## Setup

In [ ]:
import os
# give googe drive the required permission
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/MyDrive/food_recommender_assignment/")
os.getcwd()

'/content/drive/MyDrive/food_recommender_assignment'

In [ ]:
try:
  import pyspark 
except:
  !pip install pyspark==3.1.2
  import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 20.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880769 sha256=8d6f679eebb81b3a3c2daeb009ec6c3e28af52d2fd374959c56136b0a46ba648
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
try:
  import lenskit 
except:
  %pip install lenskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 74 kB 1.8 MB/s 
     |████████████████████████████████| 256 kB 10.5 MB/s 
     |████████████████████████████████| 87 kB 6.8 MB/s 


In [ ]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

In [ ]:
spark = SparkSession.builder.master("local").config('spark.ui.port', '4050').getOrCreate()

In [ ]:
spark

## Imports

In [ ]:
# import necessary libraries 
import pandas as pd
import numpy as np

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType, ArrayType

In [ ]:
# Import the required functions for ALS and estimating
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
from lenskit import batch, topn, util
from lenskit.algorithms import Recommender, als, item_knn as knn

## Read the data

To connect the data files with your google collab, download them and upload them to your home folder in your google drive. 

In [ ]:
train_ratings_df = spark.read.parquet('Dataset/train_interaction_level_df.parquet', 
                                      header=True, 
                                      inferSchema=True)

In [ ]:
test_ratings_all_df  = spark.read.parquet('Dataset/test_interaction_level_df.parquet', 
                                          header=True, 
                                          inferSchema=True)

In [ ]:
raw_recipes_df = spark.read.csv("RAW_recipes_cleaned.csv", 
                                header=True, 
                                inferSchema=True)

In [ ]:
train_ratings_df.select(F.countDistinct("user_id")).collect()[0][0]

23362

In [ ]:
train_ratings_df.select("rating").count()

74673

In [ ]:
numerator = train_ratings_df.select("rating").count()


num_users = train_ratings_df.select(F.countDistinct("user_id")).collect()[0][0]
num_recipes = train_ratings_df.select(F.countDistinct("recipe_id")).collect()[0][0]

denominator = num_users * num_recipes

sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The training dataframe is ", "%.7f" % sparsity + "% empty.")

The training dataframe is  99.9829264% empty.


## Functions

In [ ]:
def manual_recommendation_check (user_id):
  '''
  Given a user ID form the test dataset, this function will return the names of the recipes recommended to the user. 

  Initialize this function after the all_recs data frame is calculated. 

  Input user_id of a user from the test set as an integer.  
  Prints the names of recipes recommended to this user. 
  Returns nothing. 
  '''
  recs_user = all_recs_als[all_recs_als.user == user_id]
  recs_user_list = list(recs_user.item.values)
  recs_user_list = [x.item() for x in recs_user_list]
  display((raw_recipes_df.filter(F.col('id').isin(recs_user_list))
               .select("name")
               .collect()
               ))

## Model

#### Task 02 - Collaborative Filtering Model

Add the argument details in the algorithm initialization below to build the ALS model.

In [ ]:
# Create ALS model
als = ALS(userCol= 'user_id', #  name of the column for users  
          itemCol= 'recipe_id', # name of the column for recipes
          ratingCol= 'rating'  , # name of the column for ratings 
          nonnegative = True, 
          implicitPrefs = False, 
          coldStartStrategy="drop"
         )

In [ ]:
assert type(als) == pyspark.ml.recommendation.ALS

#### Training

In [ ]:
#Fit the model to the 'train' dataset
model = als.fit(train_ratings_df)

## Prediction 

In [ ]:
test_ratings_all_df.count()

18684

In [ ]:
# use the model to create predictions for test data
test_predictions_unseen = model.transform(test_ratings_all_df) 

In [ ]:
test_predictions_unseen.count()

5854

In [ ]:
test_predictions_unseen.select(F.col("user_id"),
                               F.col("recipe_id"),
                               F.col("rating"),
                               F.col("prediction")
                              ).show(5)

+-------+---------+------+----------+
|user_id|recipe_id|rating|prediction|
+-------+---------+------+----------+
| 199020|    55265|     5| 4.5994487|
| 369284|    76143|     5| 3.7438645|
| 224235|    89385|     5| 4.2358184|
| 385423|    95476|     4|  3.671374|
| 538098|    95476|     5|  4.772101|
+-------+---------+------+----------+
only showing top 5 rows



In [ ]:
# Each user in the test set must have 10 predictions. 
# Use ALS model to get these predictions. 
# You can use the recommendForAllUsers() method. 

recommendations = model.recommendForAllUsers(10) 

In [ ]:
# Transfroming the results by exploding the recommendations column

recommendations = (recommendations.select(F.col("user_id"),
                                          F.posexplode(F.col("recommendations")).alias("pos", "item")) 
                                  .select(F.col("user_id"),
                                          F.col("pos"), 
                                          F.col("item.recipe_id").alias("recomended_recipe_id"), 
                                          F.col("item.rating").alias("predicted_rating")))

## Evaluation

### Task 04 - Model Evaluation

#### 1. RMSE

Add the argument details to the evaluator function below to calculate the RMSE score of the ALS model. 

In [ ]:
# Define evaluator as RMSE and print RMSE value
evaluator_seen = RegressionEvaluator(metricName="rmse", 
                                     labelCol= 'rating',  # name of the column that has the ratings 
                                     predictionCol= 'prediction'# name of the column that has the predicted ratings. 
                                     ) 

In [ ]:
RMSE = evaluator_seen.evaluate(test_predictions_unseen ) #  evaluator to find the RMSE on the test set. 
print(RMSE)

1.4093766071748897


In [ ]:
RMSE = evaluator_seen.evaluate( ) #  evaluator to find the RMSE on the test set. 
print(RMSE)

1.4196906625188663


#### Rank based Metrics

We will use the lenskit library to calculate the ranking-based matrics. The lenskit library is available in pandas only so we need to convert the data frames from PySpark dataframes to Pandas dataframes. 

In [ ]:
all_recs_als = recommendations.toPandas()

In [ ]:
all_recs_als

,user_id,pos,recomended_recipe_id,predicted_rating
0,28170,0,166172,5.912496
1,28170,1,128716,5.843215
2,28170,2,280255,5.796579
3,28170,3,243882,5.796256
4,28170,4,225101,5.791911
...,...,...,...,...
233615,1904821,5,9410,0.000000
233616,1904821,6,9970,0.000000
233617,1904821,7,10150,0.000000
233618,1904821,8,11440,0.000000


In [ ]:

column_names = {'user_id':'user',	'pos':'rank',	'recomended_recipe_id':'item',	
                'predicted_rating':'score'} # a dictionary with current column names as keys and the intended column names as values. 
all_recs_als = all_recs_als.rename(columns=column_names)

In [ ]:
all_recs_als

,user,rank,item,score
0,28170,0,166172,5.912496
1,28170,1,128716,5.843215
2,28170,2,280255,5.796579
3,28170,3,243882,5.796256
4,28170,4,225101,5.791911
...,...,...,...,...
233615,1904821,5,9410,0.000000
233616,1904821,6,9970,0.000000
233617,1904821,7,10150,0.000000
233618,1904821,8,11440,0.000000


In [ ]:
# Adding a column to make sure the we know these recommendations are from the ALS algorithm. 

all_recs_als["algorithm"] = "ALS"

In [ ]:
test_ratings_all_df.select(F.col('user_id').alias('user'),
                               F.col('recipe_id').alias('item'),
                               F.col('rating')).show(2)

+----------+-----+------+
|      user| item|rating|
+----------+-----+------+
|1802380878|35912|     0|
|2001602879|40335|     5|
+----------+-----+------+
only showing top 2 rows



In [ ]:
test_data = (test_ratings_all_df.select(F.col('user_id').alias('user'),
                               F.col('recipe_id').alias('item'),
                               F.col('rating')).toPandas())

In [ ]:
test_data

,user,item,rating
0,1802380878,35912,0
1,2001602879,40335,5
2,2758877,50348,0
3,199020,55265,5
4,369284,76143,5
...,...,...,...
18679,855082,438292,5
18680,1553277,447699,0
18681,2000072578,447699,5
18682,2775141,469503,5


In [ ]:
# code to calculate the necessary metrics 
# the code below uses lenskit library to evaluate the rank metrics

rla = topn.RecListAnalysis()
rla.add_metric(topn.recip_rank)
rla.add_metric(topn.ndcg)
rla.add_metric(topn.dcg)
results = rla.compute(all_recs_als, test_data)
results.head()

nrecs  recip_rank  ndcg  dcg
algorithm user                                
ALS       28170      10         0.0   NaN  NaN
          56680      10         0.0   0.0  0.0
          108460     10         0.0   0.0  0.0
          139830     10         0.0   NaN  NaN
          198430     10         0.0   NaN  NaN

## Manual Prediction Checking

1. Why are ndcg and dcg nulls? EG: user 28170

In [ ]:
test_data[test_data.user == 28170]

,user,item,rating


In [ ]:
all_recs_als[all_recs_als.user == 28170]

,user,rank,item,score,algorithm
0,28170,0,166172,5.912496,ALS
1,28170,1,128716,5.843215,ALS
2,28170,2,280255,5.796579,ALS
3,28170,3,243882,5.796256,ALS
4,28170,4,225101,5.791911,ALS
5,28170,5,16768,5.784569,ALS
6,28170,6,325042,5.729640,ALS
7,28170,7,379378,5.725366,ALS
8,28170,8,89272,5.679644,ALS
9,28170,9,209280,5.678403,ALS


User ```28170``` does not appear in the test set. Hence cannot be evaluated. 

2. Why are all metrics 0 for specific users?

In [ ]:
test_data[test_data.user == 56680]

,user,item,rating
446,56680,229831,5


In [ ]:
all_recs_als[all_recs_als.user == 56680]

,user,rank,item,score,algorithm
10,56680,0,156550,6.459229,ALS
11,56680,1,201382,6.430128,ALS
12,56680,2,146185,6.143998,ALS
13,56680,3,119604,6.136050,ALS
14,56680,4,329345,6.135118,ALS
15,56680,5,319562,6.132361,ALS
16,56680,6,98791,6.114608,ALS
17,56680,7,16768,6.061062,ALS
18,56680,8,382175,6.043922,ALS
19,56680,9,283173,6.042053,ALS


User-recipe combination does not appear in the recommendations set. 

3. Are any non zero metrics? 

In [ ]:
results[results.ndcg > 0]

nrecs  recip_rank      ndcg       dcg
algorithm user                                          
ALS       1133190     10    0.200000  0.123919  1.934264
          865936      10    0.111111  0.029840  1.204120

In [ ]:
results[results.dcg > 0]

nrecs  recip_rank      ndcg       dcg
algorithm user                                          
ALS       1133190     10    0.200000  0.123919  1.934264
          865936      10    0.111111  0.029840  1.204120

In [ ]:
results[results.recip_rank > 0]

nrecs  recip_rank      ndcg       dcg
algorithm user                                          
ALS       1133190     10    0.200000  0.123919  1.934264
          865936      10    0.111111  0.029840  1.204120

There are few user recipe combination that has a corresponding value in the test set. Hence, we have only few non-zero value of evaluation metrics.


Manually, check one prediction. 

In [ ]:
manual_recommendation_check(653438)

[Row(name='big soft sour cream sugar cookies'),
 Row(name='briana s belt buster pot roast'),
 Row(name='carrot cake with lemon icing'),
 Row(name='chicken in a potato nest'),
 Row(name='chicken wings or ribs habanero hot sauce'),
 Row(name='fiesta mexican lasagna'),
 Row(name='linda s ladies town hall cheesecake'),
 Row(name='old fashioned sour cream sugar cookies'),
 Row(name='smoky spinach macaroni and cheese'),
 Row(name='spiced corn with coconut')]

Of the recipes that have been recommended, few appear similar, and few do not. 

## Saving the models and predictions

In [ ]:
all_recs_als.to_csv("recommendation_als.csv", 
                    index=False)

In [ ]:
model.save('ALS_model.model')